In [1]:
import pandas as pd 
import numpy as np
import os
from subprocess import check_output
import shutil

from rule import *
from amie import *

ERROR! Session/line number was not unique in database. History logging moved to new session 412


In [2]:
root_source = "./../../FB15k-237-num/"
store_data = "./../../Data_FB/FB15K_lte_gt.tsv"

In [3]:
f = open(root_source+"FB15K_NumericalTriples.txt", "r")
dic_predicate = {}
for line in f:
    line_split = line.split("\n")[0].split("\t")
    line_split[2] = float(line_split[2]) 
    line_split = tuple(line_split)
    if line_split[1] in dic_predicate.keys():
        dic_predicate[line_split[1]].add(line_split)
    else : 
        dic_predicate[line_split[1]] = {line_split}
f.close()

In [4]:
thresholds = np.arange(0.25,1,0.25) #[0.25, 0.50, 0.75]
thresholds_str = ["-"+(str(int(i*100))) for i in thresholds]

In [5]:
def write_file(X, f, threshold, comparator):
    f.write(f"{X['Subject']}\t{X['Predicate']}_LTE_{threshold}\t{X['Object']<=comparator}\n")
    f.write(f"{X['Subject']}\t{X['Predicate']}_GT_{threshold}\t{X['Object']>comparator}\n")

In [6]:
f = open(store_data, "w")

for key in dic_predicate.keys():
    tp_df = pd.DataFrame.from_dict(dic_predicate[key]).rename(columns={0: "Subject", 1: "Predicate", 2: "Object"})
    tp_df_describe = tp_df["Object"].quantile(thresholds)
    for threshold in thresholds: 
        tp_df.apply(write_file, args=(f, threshold, tp_df_describe[threshold]), axis=1)

f.close()

In [7]:
data = open(root_source+"train.txt", "r")
f = open(store_data, "a")

predicate_to_find = set()

for line in data: 
    f.write(line)
    predicate_to_find.add(line.split("\n")[0].split("\t")[1])
    
data.close()
f.close()

In [8]:
htr = ""
for p in predicate_to_find:
    htr+=p+","
htr = htr[:-1]
print(htr)

/people/ethnicity/languages_spoken,/film/film/country,/base/schemastaging/organization_extra/phone_number./base/schemastaging/phone_sandbox/service_language,/user/alexander/philosophy/philosopher/interests,/film/film/release_date_s./film/film_regional_release_date/film_release_region,/location/location/partially_contains,/film/film/runtime./film/film_cut/film_release_region,/education/educational_institution/colors,/american_football/football_team/current_roster./sports/sports_team_roster/position,/film/film/dubbing_performances./film/dubbing_performance/actor,/film/film/written_by,/user/ktrueman/default_domain/international_organization/member_states,/music/record_label/artist,/location/location/contains,/organization/endowed_organization/endowment./measurement_unit/dated_money_value/currency,/baseball/baseball_team/team_stats./baseball/baseball_team_stats/season,/sports/sports_league_draft/picks./sports/sports_league_draft_pick/school,/award/award_nominee/award_nominations./award/awa

In [9]:
res = check_output(f'java -jar ./../amie3.jar -htr {htr} {store_data}', shell=True)

res_parsed = parse_amie(res)

In [10]:
len(res_parsed)

13180

In [11]:
res_parsed

{?a <http://rdf.freebase.com/ns/film.film.initial_release_date>_LTE_0.5 ?f & ?b <http://rdf.freebase.com/ns/law.invention.date_of_invention>_LTE_0.5 ?f => ?a /film/film/distributors./film/film_film_distributor_relationship/film_distribution_medium ?b,
 ?b /dataworld/gardening_hint/split_to ?f & ?a /location/administrative_division/country ?f => ?a /location/administrative_division/first_level_division_of ?b,
 ?a <http://rdf.freebase.com/ns/government.legislative_session.date_ended>_LTE_0.5 ?f & ?b <http://rdf.freebase.com/ns/location.location.area>_GT_0.25 ?f => ?a /government/legislative_session/members./government/government_position_held/district_represented ?b,
 ?b /award/award_winning_work/awards_won./award/award_honor/award_winner ?f & ?f /base/saturdaynightlive/snl_cast_member/seasons./base/saturdaynightlive/snl_season_tenure/cast_members ?a => ?a /tv/tv_personality/tv_regular_appearances./tv/tv_regular_personal_appearance/program ?b,
 ?e /base/popstra/celebrity/dated./base/pops